In [48]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [49]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [50]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [51]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [52]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()


In [53]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original data
# Set the coinid column as index
df_market_data_scaled = pd.DataFrame(scaler.fit_transform(df_market_data), columns=df_market_data.columns, index=df_market_data.index)

# Display sample data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [54]:
# Create a list with the number of k-values from 1 to 11
k_values = range(1, 12)


In [55]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_market_data_scaled)
    inertia_values.append(kmeans.inertia_)


c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Pyt

In [56]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k_values, 'inertia': inertia_values}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [57]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(x='k', y='inertia', title="Elbow Curve", xlabel='Number of Clusters (k)', ylabel='Inertia')
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [58]:
# Initialize the K-Means model using the best value for k
best_k = 3
kmeans_model = KMeans(n_clusters=best_k, random_state=0)


In [59]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(df_market_data_scaled)

c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [60]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = kmeans_model.predict(df_market_data_scaled)


# Print the resulting array of cluster values.
print("Cluster labels:", clusters)


Cluster labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0
 1 1 1 1]


In [61]:
# Create a copy of the DataFrame
df_clustered = df_market_data_scaled.copy()


In [62]:
# Add a new column to the DataFrame with the predicted clusters
df_clustered['Cluster'] = clusters


# Display sample data
print(df_clustered.head())


              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

In [63]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot = df_clustered.hvplot.scatter(x='price_change_percentage_24h', 
                                           y='price_change_percentage_7d', 
                                           by='Cluster', hover_cols='index', 
                                           title="Cryptocurrency Clusters", xlabel="24h Price Change (%)", ylabel="7d Price Change (%)")
scatter_plot

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [64]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [65]:
# Print the columns of the DataFrame to check the correct name of the cluster column
print("Columns in DataFrame:", df_market_data_scaled.columns)

# Assuming the cluster column is correctly named 'Cluster', not 'Clusters'
if 'Cluster' in df_market_data_scaled.columns:
    # Drop the 'Cluster' column correctly before applying PCA
    df_for_pca = df_market_data_scaled.drop('Cluster', axis=1)
else:
    # If there is no 'Cluster' column, proceed with the original scaled data
    df_for_pca = df_market_data_scaled

Columns in DataFrame: Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')


In [66]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
df_market_data_pca = pca.fit_transform(df_for_pca)

# View the first five rows of the DataFrame. 
print(df_market_data_pca[:5])


[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]]


In [67]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
print("Explained variance by each principal component:", explained_variance)
print("Total explained variance:", explained_variance.sum())


Explained variance by each principal component: [0.3719856  0.34700813 0.17603793]
Total explained variance: 0.8950316570309842


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.895

In [68]:
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(df_market_data_pca, columns=['PC1', 'PC2', 'PC3'])

# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
df_pca['coin_id'] = df_market_data.index

# Set the coinid column as index
df_pca.set_index('coin_id', inplace=True)

# Display sample data
print(df_pca.head())


                   PC1       PC2       PC3
coin_id                                   
bitcoin      -0.600667  0.842760  0.461595
ethereum     -0.458261  0.458466  0.952877
tether       -0.433070 -0.168126 -0.641752
ripple       -0.471835 -0.222660 -0.479053
bitcoin-cash -1.157800  2.041209  1.859715


---

### Find the Best Value for k Using the PCA Data

In [69]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = range(1,12)

In [70]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
# Create an empty list to store the inertia values

for k in k_values_pca:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_pca = KMeans(n_clusters=k, random_state=0)
    
    # 2. Fit the model to the data using `df_market_data_pca`
    kmeans_pca.fit(df_market_data_pca)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values_pca.append(kmeans_pca.inertia_)

c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KatreeceHattaway\AppData\Local\Programs\Pyt

In [71]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {'k': k_values_pca, 'inertia': inertia_values_pca}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)


In [72]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x='k', y='inertia', title="Elbow Curve using PCA Data", xlabel='Number of Clusters (k)', ylabel='Inertia')
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** no

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [73]:
# Initialize the K-Means model using the best value for k
best_k_pca = 3


In [74]:
# Fit the K-Means model using the PCA data
kmeans_pca = KMeans(n_clusters=best_k_pca, random_state=0)
kmeans_pca.fit(df_market_data_pca)



c:\Users\KatreeceHattaway\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [75]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_pca = kmeans_pca.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print("Cluster labels:", clusters_pca)


Cluster labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0
 1 1 1 1]


In [76]:
# Create a copy of the DataFrame with the PCA data
df_pca_clustered = pd.DataFrame(df_market_data_pca, columns=['PC1', 'PC2', 'PC3'], index=df_market_data.index)


# Add a new column to the DataFrame with the predicted clusters
df_pca_clustered['Cluster'] = clusters_pca


# Display sample data
print(df_pca_clustered.head())


                   PC1       PC2       PC3  Cluster
coin_id                                            
bitcoin      -0.600667  0.842760  0.461595        1
ethereum     -0.458261  0.458466  0.952877        1
tether       -0.433070 -0.168126 -0.641752        1
ripple       -0.471835 -0.222660 -0.479053        1
bitcoin-cash -1.157800  2.041209  1.859715        1


In [77]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_pca_plot = df_pca_clustered.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='Cluster',
    hover_cols='index',
    title="Cryptocurrency Clusters based on PCA",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2"
)
scatter_pca_plot

:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [78]:
# Composite plot to contrast the Elbow curves

# Create the Elbow curve for original data
elbow_plot_original = df_elbow.hvplot.line(
    x='k', 
    y='inertia', 
    title="Elbow Curve for Original Data",
    xlabel='Number of clusters (k)',
    ylabel='Inertia',
    color='blue',
    width=400,
    height=400
)

# Create the Elbow curve for PCA data
elbow_plot_pca = df_elbow_pca.hvplot.line(
    x='k', 
    y='inertia', 
    title="Elbow Curve for PCA Data",
    xlabel='Number of clusters (k)',
    ylabel='Inertia',
    color='green',
    width=400,
    height=400
)

# Combine both plots side by side
composite_elbow_plot = elbow_plot_original + elbow_plot_pca

# Display the composite plot
composite_elbow_plot


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [79]:
# Composite plot to contrast the clusters
# Assuming 'df_clustered' contains the original data with cluster labels and 'df_pca_clustered' contains PCA data with cluster labels

# Scatter plot for clusters from the original data
scatter_original = df_clustered.hvplot.scatter(
    x='price_change_percentage_24h', 
    y='price_change_percentage_7d', 
    by='Cluster',
    hover_cols='index',
    title="Clusters from Original Data",
    xlabel="24h Price Change (%)",
    ylabel="7d Price Change (%)",
    width=500,
    height=500,
    colormap='Category10'
)

# Scatter plot for clusters from the PCA data
scatter_pca = df_pca_clustered.hvplot.scatter(
    x='PC1', 
    y='PC2', 
    by='Cluster',
    hover_cols='index',
    title="Clusters from PCA Data",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2",
    width=500,
    height=500,
    colormap='Category10'
)

# Combine both plots side by side
composite_cluster_plot = scatter_original + scatter_pca

# Display the composite plot
composite_cluster_plot


:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** By using fewer features to cluster the K means data, 